In [16]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from views_dataviz.map import mapper, utils
from views_dataviz import color

import sqlalchemy as sa
from ingester3.config import source_db_path

# Mapping examples

This notebook shows how the `Mapper` can be used in your projects. 

We'll first walk through some steps to illustrate the logic. You can also skip to a section that is directly relevant to you:

1. [The basics](#basics)
1. [Adding layers](#layering)
1. [Adding a colorbar](#colorbar)
1. [Further customization](#customization)
1. [Mapping presets](#presets)

For illustration, we're getting the country table from db into a GeoDataFrame limited to Africa. We then add mock random predictions and make it as if from month_id 500.

In [17]:
engine = sa.create_engine(source_db_path)
gdf = gpd.GeoDataFrame.from_postgis(
    "SELECT id, geom FROM prod.country WHERE in_africa=1", engine, geom_col='geom'
)
gdf = gdf.to_crs(4326) 
gdf["preds"] = np.random.uniform(low=0, high=1, size=len(gdf))
gdf["month_id"] = 500

Here's an example of how to generate a typical ViEWS map, using a mapping preset.

In [ ]:
from views_dataviz.map.presets import ViewsAfrica

m = ViewsAfrica(
    label=f"preds\nr_2021_08_01",
    title="2021-8",
    scale="logodds",
).add_layer(
    gdf.loc[gdf.month_id==500],
    edgecolor="black",
    linewidth=0.5,
    column="preds",
)

<a id="basics"></a>
## Basics
___
The `Mapper` class allows you to customize any layer you want, at any point of "layeredness". The following goes through the basics to illustrate this idea.

Initialize a mapper instance with `Mapper`.

In [ ]:
?mapper.Mapper

When you instantiate Mapper without any additional layers in a notebook, it returns an empty frame with a default title. The Mapper instance is set up with some overarching attributes relevant to the figure:

In [ ]:
example = mapper.Mapper(
    width=10, 
    height=10,
    bbox=[-18.5, 52.0, -35.5, 38.0],  # xmin, xmax, ymin, ymax for African mainland.
    frame_on=True,
    title="example",
)
example.__dict__

To save your map at any stage, use `save`:

In [ ]:
example.save("./empty_example.png")

<a id="layering"></a>
## Adding layers
___

We can now start adding our layers into this frame with the `add_layer` method, which takes a GeoDataFrame, a cmap (either a string, or a matplotlib ColorMap), and any keyword-arguments that can be supplied to the `plot` method on a geopandas GeoDataFrame. You can also supply `inform_colorbar` to set or override a colorbar automatically with information from the layer - this will be illustrated later. Preview the map so far with `.fig`.
> *Note: You can use any GeoDataFrame here. The Mapper does not expect any particular ids, or crs.*

In [ ]:
?mapper.Mapper.add_layer

In [ ]:
example = example.add_layer(
    gdf=gdf.loc[gdf.month_id==500],
    cmap="cividis",
    edgecolor="black",
    linewidth=0.5,
    column="preds",
)
example.fig

Stack layers simply with additional `add_layer` calls, for instance when you want to add a marker. Use `zorder` if you need the adjust the order of the layers at any point.

In [ ]:
example = example.add_layer(
    gdf=gdf.loc[(gdf.month_id==500) & (gdf.preds > .8)].geometry.centroid,
    marker="*",
    markersize=100,
    color="firebrick"
)
example.fig

<a id="colorbar"></a>
## Adding a colorbar
___

With respect to the colorbar, you have various options. One is to use the `legend` parameter when adding a layer. This however sets up a colorbar that is not equal in height to the figure frame:

In [ ]:
mapper.Mapper(
    width=10, 
    height=10,
    bbox=[-18.5, 52.0, -35.5, 38.0],  # xmin, xmax, ymin, ymax for African mainland.
    frame_on=True,
    title="example",
).add_layer(
    gdf=gdf.loc[gdf.month_id==500],
    cmap="cividis",
    edgecolor="black",
    linewidth=0.5,
    column="preds",
    legend=True
)

GeoPandas's suggested solution is to add a custom ax instead, and add a colorbar to that. The `add_colorbar` method allows you to do so. Note that in this case you have to provide the cmap, vmin, and vmax. 

In [ ]:
?mapper.Mapper.add_colorbar

In [ ]:
mapper.Mapper(
    width=10, 
    height=10,
    bbox=[-18.5, 52.0, -35.5, 38.0],  # xmin, xmax, ymin, ymax for African mainland.
    frame_on=True,
    title="example",
).add_layer(
    gdf=gdf.loc[gdf.month_id==500],
    cmap="cividis",
    edgecolor="black",
    linewidth=0.5,
    column="preds",
).add_colorbar(
    cmap="cividis", 
    vmin=gdf.loc[gdf.month_id==500, "preds"].min(),
    vmax=gdf.loc[gdf.month_id==500, "preds"].max(),
)

The `add_colorbar` method gives you full control over the colorbar. Alternatively, you can simply set a default colorbar to the min-max range of the series plotted with the `inform_colorbar` argument of the `add_layer` method:

In [ ]:
example = mapper.Mapper(
    width=10, 
    height=10,
    bbox=[-18.5, 52.0, -35.5, 38.0],  # xmin, xmax, ymin, ymax for African mainland.
    frame_on=True,
    title="example",
).add_layer(
    gdf=gdf.loc[gdf.month_id==500],
    cmap="cividis",
    edgecolor="black",
    linewidth=0.5,
    column="preds",
    inform_colorbar=True,
)

<a id="customization"></a>
## Further customization
___

### Ax

Note that you can retrieve the ax from your instance at any stage, and perform operations on and with that as usual in matplotlib. For instance:

In [ ]:
ax = example.ax
ax.set_title("custom title", size=25)
example.fig

### Colorbar

While `add_colorbar` gives control over the colorbar's customization, you may want to make changes after it's already set. You do this manually using the `cax` and `cbar` attributes, for instance:

In [ ]:
example.cbar.set_ticks([.3, .6, .9])
example.cbar.set_ticklabels(["3 USD", "6 USD", "9 USD"])

In [ ]:
example.fig

### Adding a custom title

`add_title` allows you to set a custom title directly onto the instance, which overrides the default.

In [ ]:
?mapper.Mapper.add_title

In [ ]:
example = example.add_title(title="2021-8", size=25)
example.fig

### Adding a "ViEWS textbox"

In [ ]:
?mapper.Mapper.add_views_textbox

In [ ]:
example = example.add_views_textbox(
    text="preds\nr_2021_08_01", 
    textsize=16
)
example.fig

### Basemaps

Add a basemap with `add_basemap`. This uses contextily, for details please see: https://contextily.readthedocs.io/en/latest/, with default source set to `ctx.providers.CartoDB.Voyager`.

In [ ]:
?mapper.Mapper.add_basemap

In [ ]:
example = example.add_basemap()
example.fig

<a id="presets"></a>
## Mapping presets
___

You can make mapping presets by inheriting from the `Mapper` class. Presets make it easy to share pre-defined looks between projects.

Presets are currently collected in `views_dataviz/map/presets/views.py`, and can be imported as in the example at the top of this notebook. See for instance the standard preset used for ViEWS maps focused on Africa:

In [20]:
from views_dataviz.map.presets import ViewsAfrica
?ViewsAfrica

Init signature:
ViewsAfrica(
    width=10,
    height=10,
    bbox=[-18.5, 52.0, -35.5, 38.0],
    cmap='viridis',
    frame_on=True,
    title='',
    label='',
    scale=None,
    vmin=None,
    vmax=None,
    tickparams=None,
)
Docstring:     
Inherits from Mapper. Runs methods on init that build the ViEWS defaults.

Attributes
----------
title: Add a custom str title.
label: Custom str label to add to textbox.
scale: Scale to set map to. Either "logodds", "prob", "delta", or None.
vmin: Minimum value of scale.
vmax: Maximum value of scale.
tickparams: Dictionary of custom tick parameters, by key-value pairs. For
    example: {0.05: "5%", 0.1: "10%"}.
File:           ~/opt/miniconda3/envs/viewser/lib/python3.9/site-packages/views_dataviz/map/presets/views.py
Type:           type
Subclasses:     
